In [1]:
# '''
# =====================================

# Nama    : Ari Budianto

# Analisis di buat untuk menyelesaikan masalah dan menentukan arah bisinis
# prospek penjualan seblak 
# =====================================
# '''

# Web Scraping

### Import library yang di butuhkan :

In [2]:
# import labrary
import time 
import pandas as pd
from scipy import stats
import numpy as np
from bs4 import BeautifulSoup as bs4
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Scraping

In [3]:
# Url target
url="https://www.tokopedia.com/search?navsource=&page=1&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st="

Scraping data dengan looping dan kondisi karena ada elemen-elemen yang berbeda pada product

```# Membuat list kosong untuk menyimpan data

data = []

# Melakukan \iterasi dari halaman 1 hingga 10
for i in range(1,11):  
    url = "https://www.tokopedia.com/search?navsource=&page={}&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=".format(i)
    driver = webdriver.Chrome()
    driver.get(url)

    #  menunggu driver 5 detik sampe selectornya muncul
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#zeus-root')))
    time.sleep(2)

    # Melakukan scroll untuk mengambil lebih banyak data
    for c in range(2):  # angka menunjukan scroll yang akan dilakukan
        driver.execute_script("window.scrollBy(0,250)")
        time.sleep(1)

    html = driver.page_source
    soup = bs4(html, "html.parser")

    #iterasi melalui elemen-elemen produk pada halaman
    for item in soup.findAll('div',{'class':'css-974ipl'}):    
        nama_produk = item.find('div',{'class':'prd_link-product-name'}).text
        harga = item.find('div',{'class':'prd_link-product-price css-1ksb19c'}).text    

        rtng = item.findAll('span',{'class':'prd_rating-average-text'})
        if len(rtng) > 0 :
            rating = item.find('span',{'class':'prd_rating-average-text'}).text
        else :
            rating = None

        rjl = item.findAll('span',{'class':'prd_label-integrity'})
        if len(rjl) > 0 :
            terjual = item.find('span',{'class':'prd_label-integrity'}).text
        else :
            terjual = None

        # Iterasi melalui elemen-elemen toko yang berbeda pada produk   
        for item2 in item.findAll('div', {'class' :'css-1rn0irl'}):
            toko = item2.find_all('span',{'class':'prd_link-shop-name css-1kdc32b flip'})[0].text  
            lokasi = item2.find_all('span', {'class' :'prd_link-shop-loc css-1kdc32b flip'})[0].text

            data.append(
                    (nama_produk,harga,toko,lokasi,terjual,rating)
            )

df=pd.DataFrame(data,columns=['Nama Barang','Harga Barang','Penjual','Toko','Terjual','Rating Produk'])
df = df.drop_duplicates()

driver.close()   # Menutup halaman yang sedang aktif

```

In [4]:
# Data csv 
df = pd.read_csv('scraping.csv')

# Dara Preparation

## Pandas

- Dari data scraping Tokopedia kita mendapat produk - produk seblak lengkap dengan nama barang, harga , penjual, barang terjual dan rating

- Informasi data frame memiliki 124 data(data ini akan berubah - rubah tergantung web scraping ), 6 kolom,semua data bertype object sebagian data memiliki missing value, dengan melihat informasi yang ada kita hitung dan buktikan missing value

In [5]:
# check info data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Nama Barang    124 non-null    object 
 1   Harga Barang   124 non-null    object 
 2   Penjual        124 non-null    object 
 3   Toko           124 non-null    object 
 4   Terjual        112 non-null    object 
 5   Rating Produk  110 non-null    float64
dtypes: float64(1), object(5)
memory usage: 5.9+ KB


- Informasi yang di dapat, maka di hitung jumlah missing value , kolom Terjual memiliki 12 missing value selain itu kolom rating produk memiliki 14 missing value
  *catatan: data yang di dapat dipengaruhi hasil scraping maka data missing value bisa bertambah mau pun berkurang

In [6]:
# check missing value
df.isnull().sum()

Nama Barang       0
Harga Barang      0
Penjual           0
Toko              0
Terjual          12
Rating Produk    14
dtype: int64

dari beberapa data missing value akan di handle dengan data cleaning jika  data yang tidak sesuai dengan tipe kita akan menyesuaikan data dengan mengubah tipe data yang sesuai. agar mempermudah langkah selanjutnya dalam melakukan proses perhitungan dan lain lain

In [7]:

# Pembersihan data dari karakter
df['Harga Barang'] =df['Harga Barang'].str.replace('Rp','')
df['Harga Barang'] =df['Harga Barang'].str.replace('.','')
df['Terjual'] =df['Terjual'].str.replace('Terjual','')
df['Terjual'] =df['Terjual'].str.replace('rb','000')
df['Terjual'] =df['Terjual'].str.replace('+','')
df['Terjual'] =df['Terjual'].str.replace(' ','')

Dari dataframe proses data cleaning berjalan dengan baik dan sesuai yang kita inginkan membuang karakter dan menambah angka yang 'rb' menjadi angka '000'

In [8]:
# Mengganti None menjadi nilai tertentu
df.fillna(value=0, inplace=True)

Setelah melakukan observasi dari data maka saya handling missing value dengan Merubah None menjadi nilai 0 


In [9]:
# membuat kolom pendapatan
cols_to_convert = ['Harga Barang','Terjual']
df[cols_to_convert]=df[cols_to_convert].astype('int64')
df['Rating Produk']=df['Rating Produk'].replace('[^\d.-]', '', regex=True).astype(float)
df.head()

,Nama Barang,Harga Barang,Penjual,Toko,Terjual,Rating Produk
0,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,15000,Central Seblak Nusantara,Tangerang Selatan,1000,4.9
1,kerupuk seblak pedas,10500,Sumber Bahagia snack,Magelang,0,0.0
2,SEBLAK JUARA INSTAN MASAK BASAH ASLI BANDUNG E...,22000,Rasa Juara Indonesia,Bandung,750,4.9
3,"Seblak Instant ""mBak'Well""",15000,Welly Mart,Tangerang Selatan,100,5.0
4,KERUPUK SEBLAK MENTAH ANEKA WARNA,11500,jajanangarut19,Jakarta Selatan,100,4.9


# Business Understanding/Problem Statement

Mendefinisikan Problem Statement
Mengunakan Kerangka SMART :
1. Specific : Menilai antusiasme dan penjualan produk seblak di Tokopedia untuk mendapatkan insight tentang tingkat popularitas dan animo konsumen terhadap produk seblak

2. Measurable : Mengukur angka penjualan, hrga jual, jumlah pesanan, dan ranting terhadap produk seblak untuk memahami seberapa banyak yang membeli dan mempberikan feedback yang positif 

3. Achievable : Data yang dibutuhkan dapat diambil melalui teknik web scraping dari situs Tokopedia, dan analisis dapat dilakukan dengan menggunakan keterampilan dan pengetahuan dalam analisis data.

4. Relevant : Analisis ini penting untuk menginformasikan keputusan bisnis apakah layak menjalankan skema dropship produk seblak berdasarkan minat dan kinerja penjualan yang diamati.

5. Time-bound : Analisis data akan dilakukan dalam rentang waktu 1 minggu, dan laporan hasil akan disiapkan untuk menunjang meningkatkan keberhasilan dalam business drop ship seblak.

`Problem statement`:
Dalam Usaha meningkatkan pendapatan Di Tokopedia Dengan skema drop ship, dalam skema dropship memiliki kendala menentukan produk seblak yang memiliki rata-rata penjualan tinggi dan diminati oleh pembeli, dengan ada nya masalah yang sudah ditentukan maka data di analisa penjualan produk seblak yang ada di Tokopedia untuk mendapat insight memahami konsumen terhadap produk seblak, dengan menganlisa pembelian produk ini dan kinerja penjjualanya.


# Analysis

- Proses Menghitung rata-rata,media,standar deviasi,skewness dan kurtosis Harga Barang

In [10]:
# Menghitung rata-rata,media,standar deviasi,skewness dan kurtosis
print('Output dari data rata-rata:',df['Harga Barang'].mean())
print('Output dari data Median:',df['Harga Barang'].median())
std = df['Harga Barang'].std()
print('Standard Deviation Harga barang:',std)
print(f"Skewness: {df['Harga Barang'].skew()}")
print(f"Kurtosis: {df['Harga Barang'].kurtosis()}")


Output dari data rata-rata: 22236.225806451614
Output dari data Median: 15000.0
Standard Deviation Harga barang: 21626.247422770168
Skewness: 2.1249005422493386
Kurtosis: 4.885370476863323


- Proses Menghitung rata-rata,media,standar deviasi,skewness dan kurtosis Banyak Produk Terjual

In [11]:
# Menghitung rata-rata,media,standar deviasi,skewness dan kurtosis
print('Output dari data rata-rata:',df['Terjual'].mean())
print('Output dari data Median:',df['Terjual'].median())
std = df['Terjual'].std()
print('Standard Deviation Terjual:',std)
print(f"Skewness: {df['Terjual'].skew()}")
print(f"Kurtosis: {df['Terjual'].kurtosis()}")

Output dari data rata-rata: 358.0241935483871
Output dari data Median: 80.0
Standard Deviation Terjual: 1101.2075741480953
Skewness: 6.487942408861648
Kurtosis: 50.2555161752538


- Proses Menghitung rata-rata,media,standar deviasi,skewness dan kurtosis Rating

In [12]:
# Menghitung rata-rata,media,standar deviasi,skewness dan kurtosis
print('Output dari data rata-rata:',df['Rating Produk'].mean())
print('Output dari data Median:',df['Rating Produk'].median())
std = df['Rating Produk'].std()
print('Standard Deviation Rating Produk:',std)
print(f"Skewness: {df['Rating Produk'].skew()}")
print(f"Kurtosis: {df['Rating Produk'].kurtosis()}")

Output dari data rata-rata: 4.3233870967741925
Output dari data Median: 4.9
Standard Deviation Rating Produk: 1.5540196434375841
Skewness: -2.444128744488976
Kurtosis: 4.104089786557097


Dari hasil perhitungan yang sudah dilakukan terhadapap poduk seblak di Tokopedia. penyebaran datanya kecendurangan memiliki outlier Highly Skewed dan memiliki positif skewed


kolom terjual

In [13]:
# membuat kolom terjual
df['Pendapatan']=df['Harga Barang'] * df['Terjual']
df.head()

,Nama Barang,Harga Barang,Penjual,Toko,Terjual,Rating Produk,Pendapatan
0,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,15000,Central Seblak Nusantara,Tangerang Selatan,1000,4.9,15000000
1,kerupuk seblak pedas,10500,Sumber Bahagia snack,Magelang,0,0.0,0
2,SEBLAK JUARA INSTAN MASAK BASAH ASLI BANDUNG E...,22000,Rasa Juara Indonesia,Bandung,750,4.9,16500000
3,"Seblak Instant ""mBak'Well""",15000,Welly Mart,Tangerang Selatan,100,5.0,1500000
4,KERUPUK SEBLAK MENTAH ANEKA WARNA,11500,jajanangarut19,Jakarta Selatan,100,4.9,1150000


- Dalam menjual produk sebalak kita mendapatkan Potensi pendapatan minimum dan maximum 

In [14]:
# check standar defiasi
std = df['Pendapatan'].std()
N = len(df)
low, up = stats.norm.interval(0.95,loc=df['Pendapatan'].mean(),scale=std/np.sqrt(N))
print('Lower Limit:',low)
print('Upper Limit:',up)

Lower Limit: 3102926.214023811
Upper Limit: 8022347.495653608


kita cari tahu lokasi tempat penjual dengan mengunakan data unique


In [15]:
# check unik
df['Toko'].unique()

array(['Tangerang Selatan', 'Magelang', 'Bandung', 'Jakarta Selatan',
       'Jakarta Barat', 'Kab. Bandung', 'Jakarta Pusat', 'Kab. Bogor',
       'Jakarta Timur', 'Surakarta', 'Kab. Bekasi', 'Surabaya', 'Bekasi',
       'Kab. Garut', 'Kab. Bandung Barat', 'Jakarta Utara', 'Tangerang',
       'Medan', 'Depok', 'Kab. Jember', 'Kab. Tangerang', 'Kab. Karawang',
       'Cimahi'], dtype=object)

memisahkan kota - kota yang berada dalam jabodetabek  dan non jabodetabek

In [16]:
# Maping kota
kota = {'Tangerang Selatan', 'Jakarta Barat',
       'Jakarta Selatan','Jakarta Pusat', 'Kab. Bogor','Jakarta Utara',
       'Jakarta Timur', 'Bekasi', 'Kab. Bekasi', 'Tangerang', 'Depok', 'Kab. Tangerang'
        }
jabodetabek=df[df['Toko'].isin(kota)]
jabodetabek.head()

,Nama Barang,Harga Barang,Penjual,Toko,Terjual,Rating Produk,Pendapatan
0,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,15000,Central Seblak Nusantara,Tangerang Selatan,1000,4.9,15000000
3,"Seblak Instant ""mBak'Well""",15000,Welly Mart,Tangerang Selatan,100,5.0,1500000
4,KERUPUK SEBLAK MENTAH ANEKA WARNA,11500,jajanangarut19,Jakarta Selatan,100,4.9,1150000
5,Seblak ala Kang Rafael | Seblak Viral | Seblak...,9216,vieka mart official,Tangerang Selatan,250,5.0,2304000
6,Kerupuk Seblak Viral / Kerupuk Mawar Putih Vir...,8000,Bumbukeluargakita,Jakarta Barat,12,5.0,96000


daftar kota diluar jabodetabek

In [17]:
# memisahkan kota non jabodetabek
nonjabodetabek=df[~df['Toko'].isin(kota)]
nonjabodetabek.head()

,Nama Barang,Harga Barang,Penjual,Toko,Terjual,Rating Produk,Pendapatan
1,kerupuk seblak pedas,10500,Sumber Bahagia snack,Magelang,0,0.0,0
2,SEBLAK JUARA INSTAN MASAK BASAH ASLI BANDUNG E...,22000,Rasa Juara Indonesia,Bandung,750,4.9,16500000
7,KERUPUK MENTAH SERBA 5000 BAHAN SEBLAK,6000,GK_Snack,Kab. Bandung,500,5.0,3000000
22,SEBRING KRUPUK KERUPUK SEBLAK KERING PEDAS DAU...,16000,Aydaa Snack,Surakarta,40,4.9,640000
25,GURILEM MINI 1KG Siomay Kering Toping Baso Aci...,29999,Pedas Gilaa,Surabaya,500,4.9,14999500


Perbedaan harga berdasakan wilyah
H0: μ_Jabodetabek = μ_nonJabodetabek

H1: μ_Jabodetabek ≠ μ_nonJabodetabek

Paired Test (Two-Sample Dependent Test)

Kesimpulan: Berdasarkan nilai p-value yang tinggi lebih dari 0,05 dan t-statistik yang positif, Dalam hasil uji ini, tidak ada bukti yang cukup untuk mendukung pernyataan bahwa biaya bahan baku yang berbeda di kedua lokasi menyebabkan perbedaan harga barang yang signifikan. meski perbedaan harga bahan baku diakui.

In [18]:
# Perbadningan
t_stat,p_val = stats.ttest_ind(jabodetabek['Harga Barang'],nonjabodetabek['Harga Barang'])
print('P-value:',p_val)
print('T_statistik:',t_stat)


P-value: 0.27365840168956757
T_statistik: 1.099630255427337


-  kolerasi

berdasarkan nilai kolerasi yang lemah dan p-value yang tinggi analisis tidak cukup bukti untuk mendukung kesimpulan yang ada bahwa orang suka produk lebih murah

In [19]:
# check kolerasi
corr_r, pval_p = stats.pearsonr(df['Harga Barang'], df['Terjual'])
print(f"r-correlation: {corr_r:.2f}, p-value: {pval_p}")

r-correlation: -0.10, p-value: 0.26183775264178766



Dari hasil perhitungan yang sudah dilakukan terhadapap poduk seblak di Tokopedia. penyebaran datanya kecendurangan memiliki outlier Highly Skewed dan memiliki positif skewed.
Menjual produk seblak kita mendapat pendapat minum dan masksimum,estimasi pendapatan perbulan adalah kisaran (3102926.214023811) sampai (8022347.495653608) untuk pendapat tambahan data ini menunjukan hasil yang positif untuk di jalankan.

Dengan harga bahan dan barang yang tinggi kita menganalisa harga barang di jobodetabek dan diluar jabodetabek apakah sama atau tidak di karenakan harga bahan yang berbeda.
dari data yang kita dapat harga barang jabodetabek dan di luar jabodetabek relatif sama karena tidak ada bukti yang mendukung perbedaan harga. itu kembali terhadap kualitas produk dan jenis produk.
Dalam menjual produk kita harus mempertimbangkan minat konsumen terhadapa produk salah satunya harga barang berdasarkan data yang telah di analisis tidak cukup bukti untuk mendukung kesimpulan yang ada bahwa orang suka produk lebih murah


# Conclusion

Kesimpulan :

Outlier dan distribusi data : Adanya outlier dala distribusi data menunjukan adanya variabilitas dalam performa penjualan. data memiliki kecendrungan positif skewed , yang dapat di indentifikasi bahwa sebagian besar Produk memiliki performa penjualan yang cukup baik.

Dengan mempertimbangkan hasil analisis ini, merekomendasikan agar skema dropship produk seblak dapat di pertimbangkan dengan lebih lanjut. Penjualan yang positif dan estimasi pendapatan yang cukup menjanjikan menunjukan bahwa produk ini memiliki potensi dalam bisnis dropship.
Meskipun analisis ini memberikan wawasan yang berguna, perlu di ingat bahwa data dan hasil bersifat berdasarkan data yang ada, kita tahu masih banyak keterbatasan informasi data di karenakan data yang kurang lengkap atau perlu lebih banyak data.
Dalam kesimpulnya, hasil analisa ini memberikan pandangan positif terhadap produk seblak sebagai potensi dalam skema dropship.